## Preprocessing

In [24]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [25]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'],axis=1)

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [26]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [27]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [28]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[
    application_df['APPLICATION_TYPE'].value_counts() < cutoff_value
    ].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()



APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [29]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [30]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[
    application_df['CLASSIFICATION'].value_counts() >1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [31]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[
    application_df['CLASSIFICATION'].value_counts() < cutoff_value
    ].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [32]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [33]:
# Split our preprocessed data into our features and target arrays
x = application_df.drop('IS_SUCCESSFUL', axis = 1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(x, y,test_size=0.2, random_state=42)


In [34]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])
hidden_nodes_layer1=14
hidden_nodes_layer2=7

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 14)             │           616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 7)              │           105 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             8 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 729 (2.85 KB)

 Trainable params: 729 (2.85 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
# Train the model
fit_model = nn.fit(X_train_scaled, Y_train, epochs=100)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6588 - loss: 0.6453
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7303 - loss: 0.5587
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7252 - loss: 0.5563
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7238 - loss: 0.5558
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 978us/step - accuracy: 0.7323 - loss: 0.5483
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 887us/step - accuracy: 0.7353 - loss: 0.5429
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 928us/step - accuracy: 0.7353 - loss: 0.5476
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - accuracy: 0.7277 - loss: 0.5499
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 886us/step - accuracy: 0.7300 - loss: 0.5492
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step - accuracy: 0.7308 - loss: 0.5491
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - accuracy: 0.7338 - loss: 0.5458
Epoch 12/100
858/858 ━━━━━━

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7262 - loss: 0.5582
Loss: 0.5581757426261902, Accuracy: 0.7262390851974487


In [39]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

### ATTEMPT_2:
1. Allow activation function to choose between relu, sigmoid, tanh
sigmoid is still the only option for the final layer
2. Allow number of neurons to vary from 6 to ~75
3. Allow for 1, 2, or 3 extra hidden layers (not input or output).

In [40]:
NUM_FEATURES = X_train.shape[1]
print("Features:", NUM_FEATURES)

def create_model(hp: kt.HyperParameters) -> tf.keras.models.Sequential:
    model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice("activation", ["relu", "sigmoid", "tanh"])

    # Allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int("first_units", 6, NUM_FEATURES*2, step=6),
                                    activation=activation,
                                    input_dim=NUM_FEATURES))
    
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                     6, NUM_FEATURES*2, step=6),
                                        activation=activation))
    # output layer
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    model.compile(loss="binary_crossentropy",
                  optimizer='adam',
                  metrics=["accuracy"])
    return model

tuner = kt.Hyperband(create_model,
                     objective="val_accuracy",
                     max_epochs=25,
                     overwrite=True,
                     hyperband_iterations=2)

Features: 43


In [41]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,
             Y_train,
             epochs=25,
             validation_data=(X_test_scaled, Y_test))

Trial 60 Complete [00h 00m 27s]
val_accuracy: 0.7300291657447815

Best val_accuracy So Far: 0.7314868569374084
Total elapsed time: 00h 12m 14s


In [42]:
# Get top 3 model hyperparameters and print the values
top_3_hp = tuner.get_best_hyperparameters(3)
for hparam in top_3_hp:
    print(hparam.values)

# Evaluate the top 3 models against the test dataset
top_3_model = tuner.get_best_models(3)
for model in top_3_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled, Y_test, verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

{'activation': 'relu', 'first_units': 36, 'num_layers': 3, 'units_0': 54, 'units_1': 30, 'units_2': 18, 'tuner/epochs': 25, 'tuner/initial_epoch': 9, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0043'}
{'activation': 'relu', 'first_units': 60, 'num_layers': 2, 'units_0': 48, 'units_1': 72, 'units_2': 42, 'tuner/epochs': 25, 'tuner/initial_epoch': 9, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0014'}
{'activation': 'relu', 'first_units': 84, 'num_layers': 3, 'units_0': 60, 'units_1': 60, 'units_2': 12, 'tuner/epochs': 25, 'tuner/initial_epoch': 9, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0023'}
215/215 - 0s - 2ms/step - accuracy: 0.7315 - loss: 0.5583
Loss: 0.558276891708374, Accuracy: 0.7314868569374084
215/215 - 0s - 2ms/step - accuracy: 0.7308 - loss: 0.5573
Loss: 0.5573035478591919, Accuracy: 0.730758011341095
215/215 - 0s - 2ms/step - accuracy: 0.7308 - loss: 0.5577
Loss: 0.5576838254928589, Accuracy: 0.730758011341095


In [43]:
best_tuned_model = tuner.get_best_models(1)[0]
# Evaluate the model using the test data
model_loss2, model_accuracy2 = best_tuned_model.evaluate(X_test_scaled, Y_test, verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

215/215 - 0s - 2ms/step - accuracy: 0.7315 - loss: 0.5583
Loss: 0.558276891708374, Accuracy: 0.7314868569374084


### ATTEMPT 3:
Here, I am trying to use few neurons (< number of features) with sigmoid for non input layer to see the impact on accuracy  

In [44]:
NUM_FEATURES = X_train.shape[1]
print("Features:", NUM_FEATURES)

nn3 = tf.keras.models.Sequential()

nn3.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=NUM_FEATURES))
nn3.add(tf.keras.layers.Dense(units=8, activation="sigmoid"))
nn3.add(tf.keras.layers.Dense(units=6, activation="sigmoid"))
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

display(nn3.summary())
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Features: 43


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 10)             │           440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 8)              │            88 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 6)              │            54 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 589 (2.30 KB)

 Trainable params: 589 (2.30 KB)

 Non-trainable params: 0 (0.00 B)

None

In [45]:

# Train the model
nn3.fit(X_train_scaled, Y_train, epochs=100)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6207 - loss: 0.6514
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7354 - loss: 0.5631
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 973us/step - accuracy: 0.7319 - loss: 0.5593
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7249 - loss: 0.5620
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7299 - loss: 0.5581
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 981us/step - accuracy: 0.7284 - loss: 0.5540
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7382 - loss: 0.5481
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7325 - loss: 0.5535
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7322 - loss: 0.5515
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 926us/step - accuracy: 0.7346 - loss: 0.5482
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7347 - loss: 0.5492
Epoch 12/100
858/858 ━━━━━━━━━━━━━━

In [46]:
# Evaluate the model using the test data
model_loss3, model_accuracy3 = nn3.evaluate(X_test_scaled, Y_test, verbose=2)
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

215/215 - 0s - 1ms/step - accuracy: 0.7270 - loss: 0.5587
Loss: 0.5587158799171448, Accuracy: 0.7269679307937622
